In [8]:
import pandas as pd
import torch
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from sklearn.model_selection import train_test_split
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [9]:
# Load data
train_data = pd.read_csv("/kaggle/input/cs4210-fall2023-assignment3/train_data.csv", header=None)
train_target = pd.read_csv("/kaggle/input/cs4210-fall2023-assignment3/train_target.csv", header=None)
test_data = pd.read_csv("/kaggle/input/cs4210-fall2023-assignment3/test_data.csv", header=None)

# Split the training data into train and validation sets
train_data, val_data, train_target, val_target = train_test_split(train_data, train_target, test_size=0.2, random_state=42)

# Convert data to PyTorch tensors
train_data = torch.Tensor(train_data.values).view(-1, 1, 48, 48)
train_target = torch.Tensor(train_target.values).squeeze().long()

val_data = torch.Tensor(val_data.values).view(-1, 1, 48, 48)
val_target = torch.Tensor(val_target.values).squeeze().long()

mean = torch.mean(train_data / 255.0)
std = torch.std(train_data / 255.0)

In [10]:
# Create a custom dataset class
class FacialExpressionDataset(Dataset):
    def __init__(self, data, target, transform=None):
        self.data = data
        self.target = target
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image, target = self.data[idx], self.target[idx]
        sample = {'image': image, 'target': target}

        if self.transform:
            sample['image'] = self.transform(image)

        return sample

In [11]:
# Define transformations
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor(),
    transforms.RandomHorizontalFlip(),
    transforms.Normalize(mean=[mean], std=[std])
])

# Create datasets and dataloaders
train_dataset = FacialExpressionDataset(train_data, train_target, transform=transform)
val_dataset = FacialExpressionDataset(val_data, val_target, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

In [12]:
# Build a CNN Model via Subclassing nn.Module
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.pool = nn.MaxPool2d(kernel_size=2, padding=0)
        self.fc1 = nn.Linear(128 * 6 * 6, 256)
        self.fc2 = nn.Linear(256, 3)  # Output has 3 classes: 0-Angry, 1-Happy, and 2-Neutral

    def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = self.pool(F.relu(self.bn3(self.conv3(x))))
        x = x.view(-1, 128 * 6 * 6)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Instantiate the model, loss function, and optimizer
model = CNNModel()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Learning Rate Scheduler
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

In [13]:
# Train the model
num_epochs = 20

for epoch in range(num_epochs):
    model.train()
    for batch in train_loader:
        inputs, labels = batch['image'], batch['target']
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    # Validate the model
    model.eval()
    with torch.no_grad():
        total_correct = 0
        total_samples = 0
        for batch in val_loader:
            inputs, labels = batch['image'], batch['target']
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total_correct += (predicted == labels).sum().item()
            total_samples += labels.size(0)

        accuracy = total_correct / total_samples
        print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item():.4f}, Validation Accuracy: {accuracy:.4f}')
        
    scheduler.step()

Epoch 1/20, Loss: 0.7018, Validation Accuracy: 0.6337
Epoch 2/20, Loss: 0.5522, Validation Accuracy: 0.6866
Epoch 3/20, Loss: 0.5491, Validation Accuracy: 0.7323
Epoch 4/20, Loss: 1.1940, Validation Accuracy: 0.5917
Epoch 5/20, Loss: 0.4309, Validation Accuracy: 0.7447
Epoch 6/20, Loss: 0.4841, Validation Accuracy: 0.7688
Epoch 7/20, Loss: 0.3099, Validation Accuracy: 0.7719
Epoch 8/20, Loss: 0.6518, Validation Accuracy: 0.7703
Epoch 9/20, Loss: 0.2193, Validation Accuracy: 0.7774
Epoch 10/20, Loss: 0.4318, Validation Accuracy: 0.7719
Epoch 11/20, Loss: 0.2808, Validation Accuracy: 0.7793
Epoch 12/20, Loss: 0.3579, Validation Accuracy: 0.7799
Epoch 13/20, Loss: 0.2271, Validation Accuracy: 0.7842
Epoch 14/20, Loss: 0.3124, Validation Accuracy: 0.7777
Epoch 15/20, Loss: 0.2203, Validation Accuracy: 0.7750
Epoch 16/20, Loss: 0.2296, Validation Accuracy: 0.7802
Epoch 17/20, Loss: 0.5476, Validation Accuracy: 0.7737
Epoch 18/20, Loss: 0.2710, Validation Accuracy: 0.7833
Epoch 19/20, Loss: 

In [14]:
# Convert test data to PyTorch tensor
X_test_tensor = torch.Tensor(test_data.values).view(-1, 1, 48, 48)

# Create test dataset and dataloader
test_dataset = FacialExpressionDataset(X_test_tensor, torch.zeros(len(X_test_tensor)), transform=transform)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Make predictions
model.eval()
predictions = []

with torch.no_grad():
    for batch in test_loader:
        inputs = batch['image']
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        predictions.extend(predicted.numpy())

# Save predictions in the required format
submission_df = pd.DataFrame({'Id': range(len(predictions)), 'Category': predictions})
submission_df.to_csv('submission.csv', index=False)